In [1]:
import lexmachina
from lexmachina.rest import ApiException
from pprint import pprint
import os

In [2]:
configuration = lexmachina.Configuration(
    host="https://api.lexmachina.com",
    access_token=os.environ["BEARER_TOKEN"]
)

In [3]:
api_client = lexmachina.ApiClient(configuration) 

In [4]:
fed_api = lexmachina.FederalDistrictCasesApi(api_client)

In [5]:
fed_response = fed_api.find_district_case_by_number(case_numbers=['2:23-cv-00640-JRG'], court='njd')

In [6]:
pprint(fed_response)

[DistrictCaseNumberSearchResult(total_count=1, input_case_number='2:23-cv-00640-JRG', input_court='njd', matches=[DistrictCaseNumberReference(url='https://api.lexmachina.com/district-cases/2008568306', district_case_id=2008568306, case_number='2:23-cv-00640', court='U.S. District Court for the District of New Jersey', title='BOLIVAR et al v. PERKINS')])]


In [7]:
party_api = lexmachina.PartiesApi(api_client)

In [8]:
# Just take the first result
party_response = party_api.search_parties(q='liberty mutual', page_number=1, page_size=1)

In [9]:
pprint(party_response)

PartySearchResults(total_count=1176, page=1, next_page='http://api.lexmachina.com/search-parties?q=liberty%20mutual&pageNumber=1&pageSize=1?q=liberty mutual&pageNumber=2&pageSize=1', parties=[PartyResult(name='Liberty Mutual Insurance Company', party_id=39480, url='https://api.lexmachina.com/parties/39480')])


In [10]:
party_response.parties[0].party_id

39480

In [11]:
state_api = lexmachina.StateCasesApi(api_client)

In [12]:
state_case_ids = []

In [13]:
state_case_query = {
    "parties": {
        "include_defendant": [party_response.parties[0].party_id]
    },
    "courts": {"state": "CA"},
    "pageSize": 100,
    'page': 1
}
    

In [14]:
done_paging = False
while not done_paging:
    state_query_response = state_api.query_state_cases(state_case_query)
    if state_query_response.cases:
        current_page = state_case_query['page']
        print(f'{current_page=}')
        result_case_ids = [caseref.state_case_id for caseref in state_query_response.cases]
        state_case_ids += result_case_ids
        state_case_query['page'] = current_page + 1
    else:
        print("done paging")
        print(f'all state cases has length {len(state_case_ids)}')
        done_paging=True
    

current_page=1
current_page=2
current_page=3
current_page=4
current_page=5
current_page=6
current_page=7
current_page=8
done paging
all state cases has length 774


In [15]:
get_state_case_response = state_api.get_state_case(2006280193)

In [16]:
pprint(get_state_case_response)

StateCaseData(state_case_id=2006280193, title='ARB, INC. VS. STANTON ENERGY RELIABILITY CENTER, LLC', state='California', court='Orange County Superior Court', case_no='30-2020-01160136', status=<CaseStatus.TERMINATED: 'Terminated'>, case_type=['Contracts (CA)'], case_tags=['General Jurisdiction'], dates=StateCaseDates(filed=datetime.date(2020, 9, 14), terminated=datetime.date(2022, 8, 25), trial=None), resolution=None, events=[CaseEvent(type='Filed', occurred=datetime.date(2020, 9, 14)), CaseEvent(type='Terminated', occurred=datetime.date(2022, 8, 25))], judges=[StateJudge(name='Derek W Hunt', state_judge_id=2661)], rulings=StateCaseRulingsByStatus(active=[]), law_firms=[LawFirm(name='Rutan & Tucker', law_firm_id=24625, client_party_ids=[156573])], attorneys=[Attorney(name='Philip J. Blanchard', attorney_id=17202199, law_firm_ids=[24625], clients_party_ids=[156573]), Attorney(name='Briana F. Richmond', attorney_id=169032428, law_firm_ids=[24625], clients_party_ids=[156573])], parties=

In [17]:
get_state_case_response.judges

[StateJudge(name='Derek W Hunt', state_judge_id=2661)]

In [ ]:
all_judges = []
num_cases_processed = 0
for case_id in state_case_ids:
    get_state_case_response = state_api.get_state_case(case_id)
    case_judges = [(j.name, j.state_judge_id) for j in get_state_case_response.judges]
    all_judges += case_judges
    num_cases_processed += 1
    if num_cases_processed % 50 == 0:
        print(f'Finished processing {num_cases_processed} out of {len(state_case_ids)} cases. judges has this length: {len(all_judges)}, number of unique judges: {len(set(all_judges))}')

Finished processing 50 out of 774 cases. judges has this length: 54, number of unique judges: 45
Finished processing 100 out of 774 cases. judges has this length: 107, number of unique judges: 82


In [18]:
len(all_judges)

1031

In [20]:
len(set(all_judges))

327

In [23]:
all_judges[:5]

[('Donna Marie Gunnell Garza', 982),
 ('Donald Robert Alvarez', 980),
 ('Robert P Dahlquist', 1505),
 ('Richard E L Strauss', 1392),
 ('Stephen Ira Goorvitch', 619)]

In [22]:
from collections import Counter

In [25]:
judge_counts = Counter()
for judge_info in all_judges:
    judge_counts[judge_info] += 1   

In [26]:
judge_counts.most_common(20)

[(('Katherine A Bacal', 1398), 16),
 (('Gregory W Pollack', 1421), 15),
 (('Barbara Marie Scheper', 48), 13),
 (('Richard E L Strauss', 1392), 11),
 (('Richard S Whitney', 1401), 11),
 (('Joel R Wohlfeil', 1676), 10),
 (('Nathan R Scott', 2922), 10),
 (('Timothy B Taylor', 1665), 10),
 (('Peter A Hernandez', 505), 10),
 (('Barbara Ann Meiers', 46), 10),
 (('Andre De La Cruz', 3004), 10),
 (('David Justin Cowan', 133), 9),
 (('Mark Chong Kim', 412), 9),
 (('Linda S Marks', 2700), 9),
 (('Jill Therese Feeney', 284), 9),
 (('Stephen Ira Goorvitch', 619), 8),
 (('John S Meyer', 1478), 8),
 (('Deborah C Servino', 2754), 8),
 (('James L Crandall', 3193), 8),
 (('Gary Yoshio Tanaka', 210), 8)]